## Some imports

In [1]:
import pandas as pd
import numpy as np
import numpy.linalg
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

import keras
import keras.backend as K
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping

from tqdm.notebook import tqdm
from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error
from scipy.stats import norm as norm_stat


Using TensorFlow backend.
/home/camilojd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/camilojd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/camilojd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/camilojd/.local/lib/python3.6/site-packages/tensorflo

## Creating Pandas Transapp DataFrames

In [2]:
"""
1. Patente
2. Sonda (servicio interno que identifica recorrido de manera unica) (variantes)
3. Servicio de Usuario (lo que ve el pasajero)
4. Dia y hora (UTC). Ultimo dia de Abril y ultimo día de Mayo
5. Latitud
6. Longitud
7. x (UTM)
8. y (UTM)
9. Distancia en Ruta (desde inicio hasta t)
10. Distancia a la ruta (ortogonal a la polilinea)
11. Velocidad instantanea (maquina del GPS del bus)
12. Operador (codificacion empresa)
13. Identificador de expedición (viaje de un lugar a otro)
"""

def create_gps(gps_name):
    columns = ["Patente", "GPS_COD_SINRUT", "idx_user", "Date", "LAT", "LON", "x_UTM", "y_UTM", "dist_rute", "dist_to_rute", "velocity", "idx_empresa", "idx_expedition"]
    # GPS Data
    df_gps = pd.read_csv(gps_name,header=None,delimiter=";")
    df_gps.columns = columns
    df_gps.index = df_gps["Date"]
    df_gps.index = pd.to_datetime(df_gps.index)
    return df_gps

def create_dict(dict_name):
    # Dictionary Data
    df_dict = pd.read_csv(dict_name,delimiter=";",encoding='latin-1')
    df_dict.index = df_dict["COD_SINRUT"]
    return df_dict

def create_shape(shape_name):
    # Shape Data
    df_shape = pd.read_csv(shape_name,delimiter=";")
    df_shape.index = df_shape["ROUTE_NAME"]
    return df_shape

In [3]:
def extract_bus(bus_name, is_ida, threshold_to_route, df_dict, df_data):
    if is_ida:
        cod_sr = df_dict[(df_dict['COD_USUARI'] == '315e') & (df_dict['Route_Name'].str.endswith('I'))][['Route_ID', 'Route_Name', 'COD_USUARI']]
    else:
        cod_sr = df_dict[(df_dict['COD_USUARI'] == '315e') & (df_dict['Route_Name'].str.endswith('R'))][['Route_ID', 'Route_Name', 'COD_USUARI']]
    df = df_data.set_index('GPS_COD_SINRUT').join(cod_sr, how='right').set_index('Date').sort_values(by='Date')
    return df[(df["dist_to_rute"] <= threshold_to_route)]

In [4]:
def filter_gps(df_gps, date, bus_shape, patente):
    if patente == None:
        return df_gps[(df_gps["Date"].str.contains(date)) & (df_gps["idx_user"] == bus_shape) ]
    else:
        return df_gps[(df_gps["Date"].str.contains(date)) & (df_gps["idx_user"] == bus_shape) & (df_gps["Patente"] == patente) ]

## Create DataFrames

In [5]:
# Parameters to filter
date = "2019-05-02"
bus_name = '315e'
ida = True
threshold_to_route = 100

In [6]:
# Load Data
df_data = create_gps("./data/{}.gps".format(date))
df_dict = create_dict("./data/2019-04-Diccionario_Servicios.csv")
df_shape = create_shape("./data/2019-04-ShapeRutas.csv")

In [7]:
# Select bus
df_gps = extract_bus(bus_name, ida, threshold_to_route, df_dict, df_data) 
df_gps.head()

,Patente,idx_user,LAT,LON,x_UTM,y_UTM,dist_rute,dist_to_rute,velocity,idx_empresa,idx_expedition,Route_ID,Route_Name,COD_USUARI
Date,,,,,,,,,,,,,,
2019-05-01 23:57:54,CJRB12,315eI,-33.385679,-70.691641,342656,6304677,13052,3,45,16,62,9183,315eI,315e
2019-05-01 23:57:57,FLXH99,315eI,-33.363410,-70.766278,335671,6307031,0,59,0,16,95,9183,315eI,315e
2019-05-01 23:58:01,FLXH94,315eI,-33.363583,-70.766153,335683,6307012,0,37,0,16,72,9183,315eI,315e
2019-05-01 23:58:07,FLXH10,315eI,-33.370171,-70.698638,341977,6306386,11213,1,48,16,111,9183,315eI,315e
2019-05-01 23:58:11,FLXH14,315eI,-33.355752,-70.738904,338204,6307923,3637,0,40,16,97,9183,315eI,315e


# Algorithm

In [8]:
columns_filter = ["Patente","x_UTM","y_UTM","velocity","dist_rute"]
bus_idx = df_gps['idx_user'].unique()[0]

## First step: Sample points

In [9]:
# Number of samples
n_samples = 300
n_samples = min(len(df_shape.loc[bus_idx]), n_samples)

sample_points = df_shape.loc[bus_idx].sample(n_samples,replace=True)[["X-Coordinate", "Y-Coordinate"]].reset_index().drop('ROUTE_NAME',axis=1)
print('Number of samples:', sample_points.shape[0])
sample_points.head()

Number of samples: 300


,X-Coordinate,Y-Coordinate
0,345389,6299941
1,345384,6299945
2,338181,6307925
3,345660,6299306
4,346255,6298160


## Second step: Search buses in sample points

In [10]:
# Search buses which goes through sample points
buses_sample_points = []
for i, sample in sample_points.iterrows():
    epsilon = 0.1
    df_epsilon = df_gps[(df_gps["x_UTM"] - sample[0] <= epsilon) & (df_gps["y_UTM"] - sample[1] <= epsilon)]
    if(df_epsilon.empty):
        sample_points.drop(i, inplace=True)
        continue
    #while(df_epsilon.empty):
    #    epsilon *= 10
    #    df_epsilon = df_gps[(df_gps["x_UTM"] - sample[0] <= epsilon) & (df_gps["y_UTM"] - sample[1] <= epsilon)]
    buses_sample_points.append(df_epsilon)
print('Buses in sample points:', len(buses_sample_points))
print('Number of samples left:', sample_points.shape[0])

Buses in sample points: 278
Number of samples left: 278


In [11]:
buses_patentes_points = [buses_point.set_index(['Patente', buses_point.index]).sort_index() for buses_point in buses_sample_points]

In [12]:
print(len(buses_patentes_points))
buses_patentes_points[0]

278


,,idx_user,LAT,LON,x_UTM,y_UTM,dist_rute,dist_to_rute,velocity,idx_empresa,idx_expedition,Route_ID,Route_Name,COD_USUARI
Patente,Date,,,,,,,,,,,,,
FLXH94,2019-05-02 15:47:09,315eI,-33.428738,-70.663178,345380,6299945,18811,2,48,16,78,9183,315eI,315e
FLXJ98,2019-05-02 11:44:59,315eI,-33.428737,-70.663210,345377,6299945,18809,0,48,16,92,9183,315eI,315e


## Third step: Create dist_point and target_time

In [13]:
# Select number of min counts for every travel
n_min_travel = 20

start_time = time.time()

buses_expeditions = []
# For every point and buses that arrive to point 
for point, buses_point in tqdm(zip(sample_points.values, buses_patentes_points),total=len(sample_points)):
    #buses_expeditions = []
    # For every expedition
    for idx_exp in buses_point['idx_expedition'].unique():
        patentes_expedition = buses_point[buses_point['idx_expedition'] == idx_exp].index.get_level_values(0).unique()
        # For every patent
        for patente in patentes_expedition:
            expedition = df_gps[(df_gps['idx_expedition'] == idx_exp) & (df_gps['Patente'] == patente)]
            bus_point = buses_point[(buses_point['idx_expedition'] == idx_exp) & (buses_point.index.get_level_values(0) == patente)]
            bus_point["dist_point"] = np.linalg.norm(bus_point[["x_UTM","y_UTM"]].values - point, axis=1)
            end_time_expedition = bus_point['dist_point'].idxmin()[1]
            expedition_end = expedition[expedition.index.get_level_values(0) <= end_time_expedition]
            # Check if n_min
            if expedition_end.shape[0] <= n_min_travel:
                continue
            # Distance Polyline
            expedition_end["dist_target"] = abs(expedition_end['dist_rute'].iloc[-1] - expedition_end['dist_rute'])

            # Create target time in seconds
            target = [((pd.to_datetime(expedition_end.index.get_level_values(0)[-1]) - pd.to_datetime(start_time)).seconds//60)%60 for start_time in expedition_end.index.get_level_values(0)]
            expedition_end["target_time"] = target

            if (not expedition_end['target_time'].is_monotonic_decreasing) or (not expedition_end['dist_target'].is_monotonic_decreasing):
                continue

            if (len(expedition_end)==0):
                continue
                
            buses_expeditions.append(expedition_end)
    #buses_dist_target.append(buses_expeditions)
print(time.time() - start_time)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/


440.2275252342224


# Retrieve and split data for model

In [15]:
# Normalize
norm = True
scaler_filename = "scaler_LSTM.save"

# Select Data
columns = ["x_UTM","y_UTM","velocity","dist_target","target_time"]
df_data = pd.concat(buses_expeditions)[columns]

# Separate data in features and targets
X = df_data[columns[:-1]]
y = df_data[columns[-1]]

# Split Train and Test
df_X_train, X_testval, df_y_train, y_testval = train_test_split(X, y, test_size=0.33, shuffle=False)
df_X_val, df_X_test, df_y_val, df_y_test = train_test_split(X_testval, y_testval, test_size=0.66, shuffle=False)

# Fix dimentions
y_train = np.expand_dims(df_y_train,1)
y_val = np.expand_dims(df_y_val,1)
y_test = np.expand_dims(df_y_test,1)

# Scale data
if norm:
    scaler_x = MinMaxScaler()
    X_train = scaler_x.fit_transform(df_X_train)
    X_val = scaler_x.transform(df_X_val)
    X_test = scaler_x.transform(df_X_test)

    # Save Scaler
    #joblib.dump(scaler_x, scaler_filename) 
  

In [16]:
print("df_data", df_data.shape)

df_data (884454, 5)


In [17]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (592584, 4)
y_train: (592584, 1)
X_val: (99235, 4)
y_val: (99235, 1)
X_test: (192635, 4)
y_test: (192635, 1)


## Create model

In [25]:
# Parameters for learning
input_units = 15
learning_rate = 0.0001
epochs = 500
batch_size = 500000
patience = 10
seq_length = 20 
percentile = 75

In [ ]:
# Time Sequences
data_gen_train = TimeseriesGenerator(X_train, y_train.squeeze(), length=seq_length)
data_gen_val = TimeseriesGenerator(X_val, y_val.squeeze(), length=seq_length, batch_size=batch_size)
data_gen_test = TimeseriesGenerator(X_test, y_test.squeeze(), length=seq_length, batch_size=batch_size)

# Model
lstm = Sequential()
lstm.add(LSTM(input_units, return_sequences=True, activation='relu', dropout=0.2, recurrent_dropout=0.2, 
                input_shape=(data_gen_train[0][0].shape[1], data_gen_train[0][0].shape[2])))
lstm.add(Dropout(0.2))
lstm.add(LSTM(10))
lstm.add(Dropout(0.2))
lstm.add(Dense(1, activation='relu'))

# Compilation
optim = optimizers.Adam(lr=learning_rate)
lstm.compile(loss='mse', optimizer=optim, metrics=['mae'])
# Train
earlyStop = EarlyStopping(monitor="val_loss",verbose=2,mode='auto',patience=patience,restore_best_weights=False)
lstm_history = lstm.fit_generator(data_gen_train, epochs=epochs, validation_data=data_gen_val, callbacks=[earlyStop])


Epoch 1/500
4630/4630 [==============================] - 42s 9ms/step - loss: 298.5775 - mae: 12.8901 - val_loss: 206.9477 - val_mae: 10.6550
Epoch 2/500
4630/4630 [==============================] - 42s 9ms/step - loss: 215.4419 - mae: 10.8827 - val_loss: 159.8889 - val_mae: 9.6442
Epoch 3/500
4630/4630 [==============================] - 42s 9ms/step - loss: 180.2213 - mae: 10.2802 - val_loss: 142.5911 - val_mae: 9.4908
Epoch 4/500
4630/4630 [==============================] - 42s 9ms/step - loss: 168.3078 - mae: 10.2155 - val_loss: 138.8731 - val_mae: 9.5909
Epoch 5/500
4630/4630 [==============================] - 42s 9ms/step - loss: 165.5178 - mae: 10.2672 - val_loss: 134.1696 - val_mae: 9.3416
Epoch 6/500
4630/4630 [==============================] - 43s 9ms/step - loss: 150.8791 - mae: 9.4582 - val_loss: 108.1280 - val_mae: 7.9584
Epoch 7/500
4630/4630 [==============================] - 43s 9ms/step - loss: 128.3323 - mae: 8.3954 - val_loss: 95.4622 - val_mae: 7.3524
Epoch 8/500
463

In [ ]:
# Train and val Loss
train_loss = lstm_history.history['loss']
val_loss = lstm_history.history['val_loss']
loss_fig = plt.figure()
loss_title = 'Model Train-Test Loss MSE'

loss_fig = plt.figure()
plt.plot(np.arange(1, len(train_loss) + 1), train_loss,label="train",alpha=0.5)
plt.plot(np.arange(1, len(val_loss) + 1), val_loss,label="validation",alpha=0.5)
plt.title(loss_title)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Train and val accuracy
train_loss = lstm_history.history['mae']
val_loss = lstm_history.history['val_mae']
loss_fig = plt.figure()
loss_title = 'Model Train-Test Loss MAE'

loss_fig = plt.figure()
plt.plot(np.arange(1, len(train_loss) + 1), train_loss,label="train",alpha=0.5)
plt.plot(np.arange(1, len(val_loss) + 1), val_loss,label="validation",alpha=0.5)
plt.title(loss_title)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# MC-Dropout prediction
lstm_mcdroput = K.function([lstm.inputs, K.learning_phase()], lstm.outputs)
lstm_mcdropout_pred = []
for _ in tqdm(range(100)):
    mcdropout_pred = lstm_mcdroput([data_gen_test[0][0]] + [1.])[0].squeeze()
    lstm_mcdropout_pred.append(np.pad(mcdropout_pred, (seq_length,0), 'constant', constant_values=0))
lstm_mcdropout_pred = np.array(lstm_mcdropout_pred).squeeze()

In [ ]:
# Prediction
lstm_pred = np.pad(lstm.predict_generator(data_gen_test).squeeze(), (seq_length,0), 'constant', constant_values=0)
y_pred = pd.DataFrame(np.percentile(lstm_mcdropout_pred, q=percentile, axis=0),index=df_y_test.index, columns=['pred']) 

## Evaluate model

In [ ]:
x_lim1 = 3000
x_lim2 = 6000
# Plot LSTM Prediction
plt.figure(figsize=(18,4))
plt.plot(df_y_test.values, label='true')
plt.plot(lstm_pred, label='predicted')
plt.legend(loc='best', prop={'size': 13})
plt.title('LSTM Failure Time Prediction')
plt.ylabel('target_time')
plt.xlabel('Step')
plt.xlim(x_lim1, x_lim2)
#plt.savefig('f1.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot LSTM MC-Dropout (mean +- 2*std)
plt.figure(figsize=(18,4))
plt.plot(df_y_test.values, label='true')
plt.plot(lstm_mcdropout_pred.mean(axis=0), label='predicted mean')
plt.fill_between(range(0, X_test.shape[0]), lstm_mcdropout_pred.mean(axis=0) - 2*lstm_mcdropout_pred.std(axis=0),
                 lstm_mcdropout_pred.mean(axis=0) + 2*lstm_mcdropout_pred.std(axis=0), color='orange', alpha='0.5')
plt.legend(loc='best', prop={'size': 13})
plt.title('MC-Dropout LSTM Failure Time Prediction')
plt.ylabel('target_time')
plt.xlabel('Step')
plt.xlim(x_lim1, x_lim2)
#plt.ylim(-0.05, 1.05)
#plt.savefig('f1.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot MLP MC-Dropout Percentile
plt.figure(figsize=(18,4))
plt.plot(df_y_test.values, label='true')
plt.plot(np.percentile(lstm_mcdropout_pred, q=percentile, axis=0), label='Percentile {}'.format(percentile))
plt.legend(loc='best', prop={'size': 13})
plt.title('MC-Dropout LSTM Failure Time Prediction')
plt.ylabel('target_time')
plt.xlabel('Step')
plt.xlim(x_lim1, x_lim2)
plt.savefig('mc_drop.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns

# Metrics
MSE = mean_squared_error(y_pred.values, df_y_test.values)
error = y_pred.values.squeeze() - df_y_test.values

norm_fitted = norm_stat.fit(error)

# Distribution Error Histogram
hist_fig, ax = plt.subplots()
ax = sns.distplot(error,fit=norm_stat,label= r"$(\mu,\sigma)=$ (" + str('%.3f'%norm_fitted[0]) + "," + str('%.3f'%norm_fitted[1]) + ")"  )
hist_title = 'Distribution Error Fitted using ' + ', '.join(columns) + ' as variables'+", norm="+str(norm)
plt.title(hist_title)
plt.xlabel("Prediction Error")
plt.ylabel("Count")
plt.legend()
#hist_fig.savefig("./results/MLP/"+hist_title+" norm="+str(norm)+".png", bbox_inches='tight')
plt.show()


print("Error Distribution Norm Fitted parameters (mu,sigma):", norm_stat.fit(error))
print("MSE:", MSE)